# 2 - Limpeza de dados

- **Missing values**: Não identificamos nenhum valor ausente no dataset;
- **Outliers**: Não identificamos nenhum outlier no dataset;
- **Inconsistência**: Pensamos em separar todos os dados relacionados à medidas de poluição, e indicadores de saúde em duas tabelas;
- **Padronização**: Para analisar os dados das medidas de poluição, precisamos separar por ano e estação, pois nossas hipóteses propostas utilizam a estação como parâmetro. O período de avaliação está bem variado, alguns indicadores possuem início em um ano, e terminam em outro. Outros indicadores são uma média do ano inteiro.

Há dois tipos de dados no nosso dataset, sazonais e anuais. Sazonais foram avaliados nas estações do ano (inverno e verão), já os anuais são uma média geral do ano.

Os dados foram avaliados em diferentes regiões. `Borough` por exemplo, é uma média de várias regiões, e `Citywide` é a média dos 5 `Borough`. 


média e valores absolutos, onde ambos irão variar de acordo com a `Measure` utilizada. Devido à esse problema, não podemos agrupar todos os registros por média. Alguns registros deverão ser agrupados pela soma. A ideia é eliminar a coluna `Geo Place Name`, pois iremos analisar Nova Iorque como um todo, e muitas das entradas estão calculadas por região.

Dentre os diferentes tipos de indicadores, separamos por:

**Sazonais (Verão/Inverno):**

375 - Nitrogen dioxide (NO2) \
386 - Ozonone (O3) \
365 - Fine particles (PM 2.5)

**Anuais:**

647 - Outdoor Air Toxics - Formaldehyde \
646 - Outdoor Air Toxics - Benzene
651 - Cardiovascular hospitalizations due to PM2.5 (age 40+) \
652 - Cardiac and respiratory deaths due to Ozone \
650 - Respiratory hospitalizations due to PM2.5 (age 20+) \
659 - Asthma emergency departments visits due to Ozone \
661 - Asthma hospitalizations due to Ozone \
657 - Asthma emergency department visits due to PM2.5 \
639 - Deaths due to PM2.5 \
653 - Asthma emergency departments visits due to Ozone \
655 - Asthma hospitalizations due to Ozone \
648 - Asthma emergency department visits due to PM2.5 \
644 - Annual vehicle miles traveled (cars) \
645 - Annual vehicle miles traveled (trucks) \
643 - Annual vehicle miles traveled \
642 - Boiler Emissions- Total NOx Emissions \
641 - Boiler Emissions- Total PM2.5 Emissions \
640 - Boiler Emissions- Total SO2 Emissions

In [15]:
import pandas as pd
import os

input_path = "air_quality.parquet"
out_folder = "out"

os.makedirs(out_folder, exist_ok=True)
df = pd.read_parquet("air_quality.parquet")
df.rename(columns={"Indicator ID": "IndicatorID", "Measure Info": "MeasureInfo", "Data Value": "DataValue", "Geo Type Name": "GeoTypeName"}, inplace=True)

## Preparando dataset dos indicadores de média

In [16]:
# Filtrar os indicadores de média
df_mean = df[df["IndicatorID"].isin([375, 386, 365, 347, 346])]

Agora precisamos separar todos os dados por estação do ano e ano. Ou seja, Time Period irá virar as colunas Season e Year

Casos diferentes de Time Period e como tratamos: \
    - `Summer YYYY` : Apenas separamos em duas colunas. \
    - `Winter YYYY-YY`: Como o inverno começa em um ano e termina em outro, consideramos o ano em que ele começa, então mantemos o primeiro ano. \
    - `YYYY-YYYY`: Uma cópia para cada ano do intervalo (transforma no formato `YYYY`). \
    - `YYYY | Annual Average YYYY`: Uma cópia para summer do mesmo ano, e outra para o winter do ano passado.


In [19]:
mask_season = (
    df_mean['Time Period'].str.startswith('Winter') |
    df_mean['Time Period'].str.startswith('Summer')
)

df_season = df_mean[mask_season].copy()
# Summer YYYY é simples, apenas separar em duas colunas
df_season[['Season', 'Year']] = df_season['Time Period'].str.split(' ', expand=True)

# Winter YYYY-YY, manter apenas o primeiro ano
df_season['Year'] = df_season['Year'].str.split('-', expand=True)[0]
df_season.drop(columns=['Time Period'], inplace=True)
df_season['Year'] = df_season['Year'].astype(int)


df_season = df_season.groupby(["IndicatorID", "Name", "Measure", "MeasureInfo", "GeoTypeName", "Season", "Year"], as_index=False)["DataValue"].mean()
df_season = df_season[df_season["GeoTypeName"] == "Citywide"]
df_season.to_parquet(f"{out_folder}/seasonal_air_quality.parquet", index=False)
df_season


# Pegando todos os intervalos de anos, para explodi-los
# mask_year_range = df_mean['Time Period'].str.contains(r'^\d{4}-\d{4}$', regex=True)
# df_year_range = df_mean[mask_year_range].copy()
#df_year_range[['Start Year', 'End Year']] = df_year_range['Time Period'].str.split('-', expand=True).astype(int)
# df_year_range['Year Count'] = df_year_range['End Year'] - df_year_range['Start Year'] + 1
# df_year_range['Year List'] = df_year_range.apply(
#     lambda row: list(range(row['Start Year'], row['End Year'] + 1)), axis=1
# )

# # Explode os intervalos dos anos
# df_year_range = df_year_range.explode('Year List').copy()
# df_year_range.rename(columns={'Year List': 'Year'}, inplace=True)
# df_year_range = df_year_range[['Indicator ID', 'Name', 'Measure', 'Measure Info', 'Year', 'Data Value']]

# Pegando apenas os anos individuais
# mask_year = (
#     df_mean['Time Period'].str.startswith('Annual Average')
# )
# df_year = df_mean[mask_year].copy()
# df_year['Time Period'] = df_year['Time Period'].str.replace('Annual Average ', '')
# df_year.rename(columns={'Time Period': 'Year'}, inplace=True)

# # Juntando os anos individuais com os intervalos explodidos
# df_year = pd.concat([df_individual_years, df_year_range], ignore_index=True)

# Separando ano em duas estações
# df_year_summer = df_year.copy()
# df_year_summer['Season'] = 'Summer'
# df_year_summer['Year'] = df_year_summer['Year'].astype(int)

# # O inverno é atribuido ao ano anterior
# df_year_winter = df_year.copy()
# df_year_winter['Season'] = 'Winter'
# df_year_winter['Year'] = df_year_winter['Year'].astype(int) - 1

# df_year_winter
# # Juntando todos os dados e fazendo a media de Data Value
# df_season = pd.concat([df_season, df_year_summer, df_year_winter], ignore_index=True)
# df_season = df_season.groupby(["Indicator ID", "Name", "Measure", "Measure Info", "Geo Type Name","Season", "Year"], as_index=False)["Data Value"].mean()
# df_season.rename(columns={"Indicator ID": "IndicatorID", "Measure Info": "MeasureInfo", "Data Value": "DataValue", "Geo Type Name": "GeoTypeName"}, inplace=True)
# df_season.to_parquet(f"{out_folder}/season_air_quality.parquet", index=False)
# del df_year, df_year_summer, df_year_winter, mask_season, mask_year


,IndicatorID,Name,Measure,MeasureInfo,GeoTypeName,Season,Year,DataValue
60,365,Fine particles (PM 2.5),Mean,mcg/m3,Citywide,Summer,2009,10.700000
61,365,Fine particles (PM 2.5),Mean,mcg/m3,Citywide,Summer,2010,11.830000
62,365,Fine particles (PM 2.5),Mean,mcg/m3,Citywide,Summer,2011,11.460000
63,365,Fine particles (PM 2.5),Mean,mcg/m3,Citywide,Summer,2012,10.310000
64,365,Fine particles (PM 2.5),Mean,mcg/m3,Citywide,Summer,2013,10.150000
...,...,...,...,...,...,...,...,...
340,386,Ozone (O3),Mean,ppb,Citywide,Summer,2019,30.550000
341,386,Ozone (O3),Mean,ppb,Citywide,Summer,2020,29.840000
342,386,Ozone (O3),Mean,ppb,Citywide,Summer,2021,30.176713
343,386,Ozone (O3),Mean,ppb,Citywide,Summer,2022,34.078943


## Preparando dataset dos indicadores anuais